In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini Text Single Turn Operation

## Get started

### Install Vertex AI SDK


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
from IPython.display import display, Markdown

import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK


In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash" # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

# Initialize the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=LOCATION) # https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

system_instructions = [
 "Respond in Korean",
 "Respond in structured ways"
]

# Access to the generative model.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

In [6]:
def generate(query:str, stream:bool):
    """
    Generate a response from the model.
    query :query to be sent to the model
    Returns:The generated response.
    """

    # Request body : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    # parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
    )

    return responses


In [7]:
from time import perf_counter

t1_start = perf_counter()

prompt = """
          You are a Generative AI specialist to help people understand the concept of AI.
          Explain what's the pros and cons of using AI in the following instructions.

          1. Define what's the Generative AI.
          2. What are differences between typical machine learning and Generative AI.
          3. What are the pros and cons of using Generative AI.
          4. Explain use cases of Generative AI in the manufacturing industry.
"""
stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")


Token Count : total_tokens: 105
total_billable_characters: 363


 Usage metadata:
{'prompt_token_count': 105, 'candidates_token_count': 728, 'total_token_count': 833}

 Finish reason:
1

 Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.0302734375
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.117675781
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0240478516
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.161132812
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.0981445312
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0825195312
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.0402832031
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0849609375
]


## 생성형 AI에 대한 이해: 장점과 단점

### 1. 생성형 AI란 무엇인가요?

생성형 AI는 기존 데이터를 학습하여 새로운 데이터를 생성하는 인공지능 기술입니다. 예를 들어, 텍스트 데이터를 학습하여 새로운 문장이나 이야기를 만들어내거나, 이미지 데이터를 학습하여 새로운 이미지를 생성하는 것이죠. 

### 2. 일반적인 머신러닝과 생성형 AI의 차이점은 무엇인가요?

* **일반적인 머신러닝:** 기존 데이터를 학습하여 미래를 예측하거나 분류하는 데 사용됩니다. 예를 들어, 이메일 스팸 필터링이나 주식 가격 예측 등이 있습니다.
* **생성형 AI:** 새로운 데이터를 생성하는 데 중점을 둡니다. 예를 들어, 이미지 생성, 텍스트 작성, 음악 작곡 등이 있습니다.

즉, 일반적인 머신러닝은 기존 데이터를 분석하고 이해하는 데 초점을 맞추는 반면, 생성형 AI는 새로운 데이터를 만들어내는 데 초점을 맞춥니다.

### 3. 생성형 AI를 사용하는 장점과 단점은 무엇인가요?

**장점:**

* **창의성 증진:** 새로운 아이디어와 콘텐츠를 생성하여 창의적인 영역에서 새로운 가능성을 열어줍니다.
* **생산성 향상:** 반복적인 작업을 자동화하여 시간과 노력을 절약하고 생산성을 높일 수 있습니다.
* **개인 맞춤형 경험 제공:** 사용자의 선호도에 맞춰 개인 맞춤형 콘텐츠를 생성하여 더 나은 사용자 경험을 제공합니다.

**단점:**

* **데이터 편향:** 학습 데이터에 편향이 존재할 경우 생성된 결과물에도 편향이 나타날 수 있습니다. 
* **저작권 문제:** 생성된 콘텐츠의 저작권 문제가 발생할 수 있습니다. 
* **윤리적 문제:** 생성된 콘텐츠가 악용될 가능성이 존재하며, 이는 윤리적인 문제를 야기할 수 있습니다.

### 4. 제조업에서의 생성형 AI 활용 사례는 무엇인가요?

* **제품 설계 최적화:** 생성형 AI를 활용하여 제품 디자인을 최적화하고 새로운 제품을 개발할 수 있습니다.
* **생산 공정 자동화:** 생산 공정을 자동화하여 생산성을 높이고 비용을 절감할 수 있습니다.
* **품질 관리 개선:** 생성형 AI를 활용하여 제품 품질을 검사하고 불량 제품을 예측할 수 있습니다.
* **설비 관리 효율 증진:** 설비 고장 예측 및 예방 정비를 통해 설비 관리 효율을 높일 수 있습니다.

**요약:** 생성형 AI는 새로운 혁신을 가져올 수 있는 강력한 기술이지만, 윤리적 문제와 데이터 편향 등의 문제점도 존재합니다. 생성형 AI를 사용할 때 이러한 문제점을 인지하고 적절한 해결 방안을 모색하는 것이 중요합니다. 


Time : 3.90207267400001 seconds


